# Prompt

**Goals**: You have $10k saved for capital expenses and you need to get yourself a new vehicle. Choose the best vehicle to buy and rank additional “runner up” vehicles.

**Guidelines**:
- Any combination of software use is allowed.

- Vehicles must be identifiable from the attached list.

- Deliverable must be returned in less than sixty minutes.

**Deliverable**:
- Send back a file showing: your main selection, your runner up selections; how you arrived at your data; how you arrived at your conclusions.

# Start

From the prompt, I know that I need to choose a vehicle from an online marketplace. Because there are 300+ selections to choose from, I think it will be faster if I can get the data to my local computer and filter it out based on my criteria. My plan is to create a script for scrapping the Autotrader's website. Then filter out the selections based on the data.

First, I use the link (https://www.autotrader.com/cars-for-sale/all-cars/dodge/journey/harper-woods-mi-48225?dma=&searchRadius=50&location=&marketExtension=inclu...) that is included in the PDF file to access the exact Autotrader's car selections. Then I modify my python's web scrapping automation script (using "beautiful soup") from my previous project (https://github.com/chilam27/Boston_Housing_Prices/blob/master/P02_DataCollection.py) to scrape all of the data that I need to my laptop. Below are the codes:

## Data Scraping

In [185]:
# Import modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [316]:
start_time = time.time() # start timer

# Setting up
def get_headers():
    headers = {'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-language':'en-US,en;q=0.9',
            'cache-control':'max-age=0',
            'upgrade-insecure-requests':'1',
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}
    return headers


result = requests.get("https://www.autotrader.com/cars-for-sale/all-cars/dodge/journey/harper-woods-mi-48225?dma=&searchRadius=50&location=&marketExtension=inclu...&isNewSearch=false&showAccelerateBanner=false&sortBy=relevance&numRecords=25", headers= get_headers())
soup = BeautifulSoup(result.content, 'lxml')

# Defining lists
car_id = [] # id of unique cars
car_name = [] # title of the listing
price = [] # price of the listing
miles = [] # miles states on the listing
color = [] # color of the car
kbb_rating = [] # kbb rating of the car


# Creating variables
page = 1
record = 0
 
total_page = 13 # there are 13 pages to date

# Scraping automation
while(page < (total_page + 1)):
    print('page: ', page, ', length:', len(car_id)) # printing the page let us know that the data is being scraped
    for web_panel in soup.find_all('div', attrs = {'class' : 'row margin-horizontal-0 padding-horizontal-0'}):
        for car_list in web_panel.find_all('div', attrs = {'class' : 'inventory-listing cursor-pointer panel panel-default'}):
            if car_list is not None:
                car_id.append(car_list.attrs['id'])
            else:
                car_id.append(None)

        for id_num in car_id[-25:]:
            for single_car in web_panel.find_all('div', attrs = {'id' : id_num}):
                for car_title in single_car.find_all('h2', attrs = {'class' : 'text-bold text-size-400 text-size-sm-500 link-unstyled'}):
                    if car_title is None:
                        car_name.append(None)
                    else:
                        car_name.append(car_title.text)

                car_price = single_car.find('span', attrs = {'class' : 'first-price'})
                if car_price is None:
                    price.append(None)
                else:
                    price.append(car_price.text)

                car_miles = single_car.find('div', attrs = {'class' : 'item-card-specifications col-xs-9 margin-top-4 text-subdued-lighter margin-bottom-3'})
                car_miles = car_miles.find('div', attrs = {'class' : 'text-bold'})
                if car_miles is None:
                    miles.append(None)
                else:
                    car_miles = car_miles.text.split(' ')[0]
                    miles.append(car_miles)

                car_color = single_car.find('div', attrs = {'class' : 'item-card-specifications col-xs-9 margin-top-4 text-subdued-lighter margin-bottom-3'})
                car_color = car_color.find('span', attrs = {'class' : 'text-gray-base text-size-200 text-size-sm-300'})
                if car_color is None:
                    color.append(None)
                else:
                    car_color = car_color.text.replace('Color: ', '')
                    color.append(car_color)

                car_kbb = single_car.find('span', attrs = {'class' : 'text-bold text-subdued'})
                if car_kbb is None:
                    kbb_rating.append(None)
                else:
                    kbb_rating.append(car_kbb.text)
    
    page += 1
    record += 25
    html = 'https://www.autotrader.com/cars-for-sale/all-cars/dodge/journey/harper-woods-mi-48225?dma=&searchRadius=50&location=&marketExtension=inclu...&isNewSearch=false&showAccelerateBanner=false&sortBy=relevance&numRecords=25&firstRecord=' + str(record) # switch to the next review page
    
    result = requests.get(html, headers= get_headers())
    soup = BeautifulSoup(result.content, 'lxml')
    
print('Time to process', len(car_id), 'rows of data:', round(time.time() - start_time,2), 'seconds') # end timer
    
# Create a dataframe based on lists above  
df = pd.DataFrame(list(zip(car_id, car_name, price, miles, color, kbb_rating)), 
                  columns = ['listing_id', 'listing_title', 'listing_price', 'miles', 'car_color', 'kbb_rating'])

page:  1 , length: 0
page:  2 , length: 25
page:  3 , length: 50
page:  4 , length: 75
page:  5 , length: 100
page:  6 , length: 125
page:  7 , length: 150
page:  8 , length: 175
page:  9 , length: 200
page:  10 , length: 225
page:  11 , length: 250
page:  12 , length: 275
page:  13 , length: 300
Time to process 322 rows of data: 18.19 seconds


Below is the result of the scraping script from above (showing only the first 25 rows of data, each row represents each selection)! As we can see that it took my code less than 20 seconds to run through 300+ selections from the site. The feautures that I scraped are:
- "listing_id" : the unique id of each listing.
- "listing_title" : the title of each listing.
- "lising_price": the price of each listing.
- "miles": miles of the car.
- "car_color": color of the car.
- "kbb_rating": Kelly Blue Book's car rating.

In [321]:
df.head(25)

,listing_id,listing_title,listing_price,miles,car_color,kbb_rating
0,578615598,Used 2018 Dodge Journey FWD SE w/ Blacktop Pac...,16000,"24,570",Black,4.4
1,578945601,Used 2019 Dodge Journey AWD GT,21990,"11,929",Silver,4.3
2,578945638,Used 2019 Dodge Journey AWD GT,22490,"10,267",Silver,4.3
3,575513977,Used 2018 Dodge Journey AWD GT,19599,"16,176",Black,4.6
4,571462155,Used 2018 Dodge Journey AWD GT,18847,"25,448",Silver,4.6
5,571462151,Used 2018 Dodge Journey AWD GT,18967,"22,517",Red,4.6
6,573704948,Used 2018 Dodge Journey AWD GT,18967,"22,517",Red,4.6
7,578188463,Used 2018 Dodge Journey AWD GT,18952,"23,960",Gray,4.7
8,574527035,Used 2018 Dodge Journey AWD GT,20320,"15,831",Gray,4.7
9,575869296,Certified 2018 Dodge Journey FWD GT,17595,"18,781",Silver,4.3


## Data Cleaning

In this section, I performed some data cleaning so it can be ready to analyze.

In [329]:
for i in range(len(df)):
    if (df.listing_price[i] is not None) & ('MSRP' in df.listing_price[i]):
        df.listing_price[i] = None
    if df.listing_price[i] is not None:
        df.listing_price[i] = df.listing_price[i].replace(',','')
        
df.listing_price = df.listing_price.astype(float)
        
df.miles = df.miles.apply(lambda x: x.replace(',','') if x is not None else None)
df.miles = df.miles.astype(float)

df.kbb_rating = df.kbb_rating.astype(float)

# Selecting Vehicle to Buy

Now comes the vehicle selection part (or the selection filtering part). Because it states in the prompt that we only have $10,000 to spend. Based on the result below, there are 73 cars I can choose that are below the price point!

In [331]:
len(df[df.listing_price <= 10000])

73

Let's assume that not only I want to choose a car that is below $10,000 , I also want to choose a car that has KBB's rating that is above 4.6 and it needs to have less than 120,000 miles. It would be really nice if the color of my car is either black or blue. Even though there may be more than one car that fit all of my criterias, I would want to choose the cheapest one out of the bunch. So the selection will be sorted (pioritize) based on the price as well.

Based on my criteria above, the best car for me is the "Used 2010 Dodge Journey 2WD SXT" (id: 575756431)! The three runner ups are "Used 2010 Dodge Journey 2WD SXT" (id: 577366924), "Used 2013 Dodge Journey FWD SXT", and "Used 2013 Dodge Journey FWD Crew". 

In [345]:
df_option = df[(df.listing_price <= 10000) & (df.kbb_rating > 4.6) & (df.miles < 120000) & 
               ((df.car_color == 'Black') | (df.car_color == 'Blue'))]
df_option.sort_values(by=['listing_price', 'miles'])

,listing_id,listing_title,listing_price,miles,car_color,kbb_rating
162,575756431,Used 2010 Dodge Journey 2WD SXT,5996.0,114000.0,Black,4.9
68,577366924,Used 2010 Dodge Journey 2WD SXT,6900.0,66000.0,Blue,4.8
190,568236548,Used 2013 Dodge Journey FWD SXT,8995.0,89048.0,Black,4.9
69,576213083,Used 2013 Dodge Journey FWD Crew,9900.0,76000.0,Blue,4.8
